<table style="width:100%">
  <tr>
    <td style="background-color:#EBF5FB; border: 1px solid #CFCFCF">
      <b>National generation capacity: Check notebook</b>
      <ul>
          <li><a href="main.ipynb">Main notebook</a></li>
          <li><a href="processing.ipynb">Processing notebook</a></li>
          <li>Check notebook (this)</li>
      </ul>
      <br>This Notebook is part of the <a href="http://data.open-power-system-data.org/national_generation_capacity">National Generation Capacity Datapackage</a> of <a href="http://open-power-system-data.org">Open Power System Data</a>.
    </td>
  </tr>
</table>

# Table of Contents
* [1. Introductory notes](#1.-Introductory-notes)
* [2. Script setup](#2.-Script-setup)
* [3. Import of processed data](#3.-Import-of-processed-data)
* [4. Visualisation of results for different energy source levels](#4.-Visualisation-of-results-for-different-energy-source-levels)
	* [4.1 Energy source level 1](#4.1-Energy-source-level-1)
		* [4.1.1 Table](#4.1.1-Table)
		* [4.1.2 Bokeh chart](#4.1.2-Bokeh-chart)
	* [4.2 Energy source level 2](#4.2-Energy-source-level-2)
		* [4.2.1 Table](#4.2.1-Table)
		* [4.2.2 Bokeh chart](#4.2.2-Bokeh-chart)
	* [4.3 Energy source level 3](#4.3-Energy-source-level-3)
		* [4.3.1 Table](#4.3.1-Table)
		* [4.3.2 Bokeh chart](#4.3.2-Bokeh-chart)
	* [4.4 Technology level](#4.4-Technology-level)
		* [4.4.1 Table](#4.4.1-Table)
		* [4.4.2 Bokeh chart](#4.4.2-Bokeh-chart)
* [5. Comparison of total capacity for energy source levels](#5.-Comparison-of-total-capacity-for-energy-source-levels)
	* [5.1 Calculation of total capacity for energy source levels](#5.1-Calculation-of-total-capacity-for-energy-source-levels)
	* [5.2 Identifcation of capacity differences for energy source levels](#5.2-Identifcation-of-capacity-differences-for-energy-source-levels)


# 1. Introductory notes

The notebook extends the [processing notebook](processing.ipynb) to make visualisations.

# 2. Script setup

In [1]:
import os.path
import math
import functions.plots as fp # predefined functions in extra file

import bokeh.plotting as plo
from bokeh.io import show, output_notebook
from bokeh.layouts import row, column
from bokeh.models import Panel, Tabs
from bokeh.models.widgets import RangeSlider, MultiSelect, Select
output_notebook()

Loading BokehJS ...

# 3. Data import

In [2]:
data_file = 'national_generation_capacity_stacked.csv'
filepath = os.path.join('output', data_file)
data = fp.load_opsd_data(filepath)
data.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
14 Nov 2019 14:03:38 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


,technology,source,source_type,weblink,year,type,country,capacity_definition,capacity,comment,energy_source_level_0,energy_source_level_1,energy_source_level_2,energy_source_level_3,technology_level,source_new
ID,,,,,,,,,,,,,,,,
0,Fossil fuels,e-control,Regulatory Authority,https://www.e-control.at/documents/1785851/181...,2014,Installed capacity in MW,AT,Gross capacity,7.243634,NaN,False,True,False,False,False,National source
1,Fossil fuels,e-control,Regulatory Authority,https://www.e-control.at/documents/1785851/181...,2015,Installed capacity in MW,AT,Gross capacity,7.059065,NaN,False,True,False,False,False,National source
2,Fossil fuels,e-control,Regulatory Authority,https://www.e-control.at/documents/1785851/181...,2016,Installed capacity in MW,AT,Gross capacity,7.323000,NaN,False,True,False,False,False,National source
3,Fossil fuels,e-control,Regulatory Authority,https://www.e-control.at/documents/1785851/181...,2017,Installed capacity in MW,AT,Gross capacity,6.469000,NaN,False,True,False,False,False,National source
4,Fossil fuels,e-control,Regulatory Authority,https://www.e-control.at/documents/1785851/181...,2018,Installed capacity in MW,AT,Gross capacity,6.492000,NaN,False,True,False,False,False,National source


# 4. Create interactive plot

Select individual width and height that fits your jupyter notebook settings.

In [3]:
width = 1000
height = 500

In [4]:
def comparison_plot(doc):
    
    # init of 5 plots for each energy level
    sources = []
    plots = []

    for level in fp.energy_levels:
        # init plots with the predfined function
        s, p = fp.init_plot(data, level, size=(width, height))
        sources.append(s)
        plots.append(p)

    # associate each plot with a tab of the interactive plot
    panels= []
    for p, level in zip(plots, fp.energy_levels):
        panels.append(Panel(child=p, title=level))

    tabs = Tabs(tabs=panels, tabs_location='below', active=2)

    # Range slider for available years
    oldest_year = min(data["year"])
    newest_year = max(data["year"])
    y_slider = RangeSlider(title="Years",
                           value=(2015,2016),
                           start=oldest_year,
                           end=newest_year,
                           step=1)

    # Select field for sources
    m_select = MultiSelect(title="Available Sources:", 
                           value=fp.global_sources,
                           options=[(s,s) for s in fp.global_sources])

    # Select button for countries
    countries = list(data["country"].unique())
    c_select = Select(title="Country", options=countries, value='FR')

    # catch all widgets 
    wid = [c_select, y_slider, m_select]
    rows = row(wid)

    # update function for `on_change` trigger
    def update(attrname, old, new):

        y = y_slider.value
        y_range = [x for x in range(y[0],y[1]+1)]
        s_selected = m_select.value
        co = c_select.value

        # run update for each plot
        for p, s, l in zip(plots, sources, fp.energy_levels):
            df = fp.filter_data_set(data, co, y_range, s_selected, l)
            source_data, x_axis = fp.prepare_data(df)
            s.data = source_data
            p.x_range.factors = x_axis

    # associate `update` function with each widget to apply updates for each change
    for w in wid:
        w.on_change('value', update)

    
    layout = column(rows, tabs)
    doc.add_root(layout)


After the bokeh plot is set up a bokeh server is started to make the plot interactive.

Possible options:

    - Select a country from the dropdown menu in the top left
    - Select a range of years from the range slider
    - Multiselect the sources you want to compare in the top right
    - Choose which "energy level" you want to investigate with the tabs below the plot

In [7]:
show(comparison_plot)

14 Nov 2019 14:06:44 - bokeh.server.server - INFO - Starting Bokeh server version 1.4.0 (running on Tornado 6.0.3)
14 Nov 2019 14:06:44 - bokeh.server.tornado - INFO - User authentication hooks NOT provided (default user enabled)


14 Nov 2019 14:06:45 - tornado.access - INFO - 200 GET /autoload.js?bokeh-autoload-element=9340&bokeh-absolute-url=http://localhost:52252&resources=none (::1) 334.56ms
14 Nov 2019 14:06:45 - tornado.access - INFO - 101 GET /ws?bokeh-protocol-version=1.0&bokeh-session-id=sPTffVKfoJwVzYv5fTWArvKiWMRX6V8fFIzJKQQYSJXY (::1) 0.00ms
14 Nov 2019 14:06:45 - bokeh.server.views.ws - INFO - WebSocket connection opened
14 Nov 2019 14:06:45 - bokeh.server.views.ws - INFO - ServerConnection created
